### Description:
- Read in the decile-binned values of labs and vitals `2_8_binned_labs_vitals`
- Read in the data `order_code_counts` from sql order counts, combine Microbiology Culture with Microbiology 
- Combine this featurized data for full complex data modeling
- Create a smaller random sample for testing 

**Notations**:
- train vs val OR train_val vs test:
    - `..._train`: used the training set (2015 - 2017) for value distributions
    - `..._test`: used the training and validation set (2015 - 2018) for value distributions   
- cohorts: 
    - `...coh4...` used 1_4_cohort
    - `...coh5...` used 1_5_cohort_final
    
**Inputs**: 
- `2_8_binned_labs_vitals`: from feature_values, binned into deciles
- `order_code_counts`: from SQL querried counts for different orders
    
**Outputs**: 
- have demo, vitals and labs in binned counts, order counts: long format, with year extracted
- `2_9_features_all_long_year`:
    - 2015 - 2017 as trained data for value distributions to bin validation data of 2018
        - used for training and select model hyperparameters based on validation data
    - test data > 2018 (2019 and some 2020) left unused
    - after training and selecting hyperparameters, re-do value distributions on 2015 - 2018 (train + val) data
    - test data > 2018 (2019 and some 2020) are now binned on based on these distributions
    - binned test data is used for the final prediction and evaluation of model performance
    
**Notes**:
- `coh4_order_code_counts` is a superset of `coh5_order_code_counts` including all 1_4_cohort
- but `2_9_coh4_features_all_long_year` is NOT a superset of `2_9_coh5_features_all_long_year` --> run them separately

In [145]:
library(data.table)
library(tidyverse)
library(lubridate)
# library(Matrix)
# library(slam)
# library(bit64)
# library(mtools) for one hot coder, not available on Nero or use caret or tidyr
options(repr.matrix.max.rows=200, repr.matrix.max.cols=30)

In [1]:
# install.packages(c("dbplyr", "RSQLite"))

Installing packages into ‘/home/jupyter/.R/library’
(as ‘lib’ is unspecified)

also installing the dependency ‘plogr’




In [146]:
datadir = "../../DataTD"
cohortdir = "../../OutputTD/1_cohort"
featuredir = "../../OutputTD/2_features"

In [147]:
cohort5 <- read.csv(file.path(cohortdir,  '1_5_cohort_final.csv'))
nrow(cohort5) #41366
cohort4 <- read.csv(file.path(cohortdir,  '1_4_cohort.csv'))
nrow(cohort4) #43980

nrow(cohort4 %>% filter(year(admit_time) < 2018)) # coh4 vs coh5 22030 vs 20762
nrow(cohort4 %>% filter(year(admit_time) == 2018)) # coh4 vs coh5 9532 vs 8929
nrow(cohort4 %>% filter(year(admit_time)> 2018)) # coh4 vs coh5 12418 vs 11675
nrow(cohort4 %>% filter(year(admit_time) < 2019)) # coh4 vs coh5 31562 vs 29691

[1] 41366

[1] 43980

[1] 22030

[1] 9532

[1] 12418

[1] 31562

In [148]:
# this one used 1_4_cohort to query orders
orders4 <- read.csv(file.path(datadir, 'coh4_order_code_counts.csv'))
nrow(orders4) #6086852

[1] 6086852

In [149]:
head(orders4)

,anon_id,pat_enc_csn_id_coded,admit_time,feature_type,features,values
,<chr>,<dbl>,<chr>,<chr>,<chr>,<int>
1,JCe78a06,131062667066,2015-01-02 01:01:00+00:00,Diagnosis,R19.7,1
2,JCe78a06,131062667066,2015-01-02 01:01:00+00:00,Diagnosis,R40.4,1
3,JCe78a06,131062667066,2015-01-02 01:01:00+00:00,Diagnosis,Z23,1
4,JCe78a06,131062667066,2015-01-02 01:01:00+00:00,Meds,"EPINEPHRINE 1 MG/ML (1:1,000) (1ML) INJ SOLN",1
5,JCe78a06,131062667066,2015-01-02 01:01:00+00:00,Diagnosis,J81.0,1
6,JCe78a06,131062667066,2015-01-02 01:01:00+00:00,Meds,CEFTRIAXONE 1 GRAM/50 ML MINI-BAG PLUS,1


In [150]:
orders4 %>% group_by(feature_type) %>% count(sort=TRUE)

feature_type,n
<chr>,<int>
Diagnosis,2834516
Meds,1868195
Lab,993484
Imaging,299944
Procedures,51696
Microbiology Culture,37374
Microbiology,1643


In [151]:
orders4_micro <- orders4 %>% mutate(feature_type = ifelse(str_detect(feature_type, "Microbiology"), "Microbiology", feature_type))
orders4_micro %>% group_by(feature_type) %>% count(sort=TRUE)

feature_type,n
<chr>,<int>
Diagnosis,2834516
Meds,1868195
Lab,993484
Imaging,299944
Procedures,51696
Microbiology,39017


In [152]:
length(unique(orders4_micro$pat_enc_csn_id_coded)) # 43970 there are 10 people who had no orders

[1] 43970

In [153]:
setdiff(orders4_micro$pat_enc_csn_id_coded, cohort4$pat_enc_csn_id_coded) # 0 
setdiff(cohort4$pat_enc_csn_id_coded, orders4_micro$pat_enc_csn_id_coded) # 10

numeric(0)

[1] 131250899044 131232385768 131143463175 131244907345 131193906942
 [6] 131239369230 131236364892 131179268110 131282120209 131239885415

In [154]:
# orders5 is only a subset of orders4 excluding obs without labs (1_5_cohort_final)
orders5_micro <- orders4_micro %>% filter(pat_enc_csn_id_coded %in% cohort5$pat_enc_csn_id_coded)
nrow(orders5_micro) #5730678
length(unique(orders5_micro$pat_enc_csn_id_coded)) #41366

[1] 5730678

[1] 41366

### Check the featurized binned labs_vitals and combine all features
- Keep them both (`..._train` and `..._test`) in the same dataframe
- Run `2_7_feature_values`, `2_8_binned_labs_vitals_train`, and `2_8_binned_labs_vitals_test` **separately** for coh4 and coh5 when combining

In [155]:
# check all features with values
values <- read.csv(file.path(featuredir, '2_7_coh5_feature_values.csv'))
nrow(values) # coh4 vs coh5: 3085046 vs 3012942
length(unique(values$pat_enc_csn_id_coded)) # 43980 vs 41366
values %>% group_by(feature_type) %>% count()

[1] 3012942

[1] 41366

feature_type,n
<chr>,<int>
demo,703222
labs,1358669
vitals,951051


In [156]:
# check train set
trainbin <- values %>% filter(year(admit_time) %in% c(2015, 2016, 2017))
testbin <- values %>% filter(year(admit_time) %in% c(2015, 2016, 2017, 2018))

nrow(trainbin %>% filter(feature_type != 'demo')) #coh4 vs coh5 1254522 vs 1240798 (check against 2.8_)
nrow(testbin %>% filter(feature_type != 'demo')) #coh4 vs coh5 1714603 vs 1694667 (check against 2.8_)

[1] 1240798

[1] 1694667

In [158]:
# read in train_binned labs and vitals
trainbinned_labs_vitals <- read.csv(file.path(featuredir, '2_8_coh5_binned_labs_vitals_train.csv'))
nrow(trainbinned_labs_vitals) #coh4 vs coh5: 1846044 vs 1825075
length(unique(trainbinned_labs_vitals$pat_enc_csn_id_coded)) #43980 vs 41366
trainbinned_labs_vitals %>% group_by(feature_type) %>% count()

[1] 1825075

[1] 41366

feature_type,n
<chr>,<int>
labs_results_train,1304104
vitals_train,520971


In [159]:
# read in test_binned labs and vitals
testbinned_labs_vitals <- read.csv(file.path(featuredir, '2_8_coh5_binned_labs_vitals_test.csv'))
nrow(testbinned_labs_vitals) #coh4 vs coh5 1849438 vs 1827530
length(unique(testbinned_labs_vitals$pat_enc_csn_id_coded)) # 43980 vs 41366
testbinned_labs_vitals %>% group_by(feature_type) %>% count()

[1] 1827530

[1] 41366

feature_type,n
<chr>,<int>
labs_results_test,1304320
vitals_test,523210


In [160]:
head(trainbinned_labs_vitals)
head(testbinned_labs_vitals)

,anon_id,pat_enc_csn_id_coded,admit_time,feature_type,features,values
,<chr>,<dbl>,<chr>,<chr>,<chr>,<int>
1,JC29f8ad2,131274729058,2019-08-31 12:52:00+00:00,labs_results_train,ALB_3,1
2,JC29f8ad2,131274729058,2019-08-31 12:52:00+00:00,labs_results_train,ALK_7,1
3,JC29f8ad2,131274729058,2019-08-31 12:52:00+00:00,labs_results_train,ALT_0,1
4,JC29f8ad2,131274729058,2019-08-31 12:52:00+00:00,labs_results_train,AST_1,1
5,JC29f8ad2,131274729058,2019-08-31 12:52:00+00:00,labs_results_train,AnionGap_9,1
6,JC29f8ad2,131274729058,2019-08-31 12:52:00+00:00,labs_results_train,BUN_8,1


,anon_id,pat_enc_csn_id_coded,admit_time,feature_type,features,values
,<chr>,<dbl>,<chr>,<chr>,<chr>,<int>
1,JC29f8ad2,131274729058,2019-08-31 12:52:00+00:00,labs_results_test,ALB_3,1
2,JC29f8ad2,131274729058,2019-08-31 12:52:00+00:00,labs_results_test,ALK_7,1
3,JC29f8ad2,131274729058,2019-08-31 12:52:00+00:00,labs_results_test,ALT_0,1
4,JC29f8ad2,131274729058,2019-08-31 12:52:00+00:00,labs_results_test,AST_1,1
5,JC29f8ad2,131274729058,2019-08-31 12:52:00+00:00,labs_results_test,AnionGap_9,1
6,JC29f8ad2,131274729058,2019-08-31 12:52:00+00:00,labs_results_test,BUN_8,1


In [161]:
# get demographics features
demos <- values %>% filter(feature_type == 'demo') %>% select(anon_id, pat_enc_csn_id_coded, admit_time, 
                                                              feature_type, features, values)
nrow(demos) # coh4 747660 vs 703222
length(unique(demos$pat_enc_csn_id_coded)) # 43980 vs 41366

[1] 703222

[1] 41366

In [162]:
# check overlapping cohort in features with values vs with order counts: should be the same, no diff
length(setdiff(trainbinned_labs_vitals$pat_enc_csn_id_coded, orders4_micro$pat_enc_csn_id_coded))
length(setdiff(orders4_micro$pat_enc_csn_id_coded, trainbinned_labs_vitals$pat_enc_csn_id_coded)) # 10 vs 2604

length(setdiff(testbinned_labs_vitals$pat_enc_csn_id_coded, orders4_micro$pat_enc_csn_id_coded))
length(setdiff(orders4_micro$pat_enc_csn_id_coded, testbinned_labs_vitals$pat_enc_csn_id_coded)) # 10 vs 2604

[1] 0

[1] 2604

[1] 0

[1] 2604

### Use orders4_micro vs orders5_micro for coh4 vs coh5

In [169]:
# combine all features for the complex data set, remove time column since demo doesn't have it
features_all <- bind_rows(demos, trainbinned_labs_vitals, testbinned_labs_vitals, orders5_micro)
nrow(features_all) #10529994 vs 10086505 (this include both train and test)
length(unique(features_all$pat_enc_csn_id_coded)) #43980 vs 41366
length(unique(features_all$features)) #42809 vs 42207
features_all %>% group_by(feature_type) %>% count(sort=TRUE)

[1] 10086505

[1] 41366

[1] 42207

feature_type,n
<chr>,<int>
Diagnosis,2680945
Meds,1747295
labs_results_test,1304320
labs_results_train,1304104
Lab,932873
demo,703222
vitals_test,523210
vitals_train,520971
Imaging,282646


In [167]:
head(features_all)

,anon_id,pat_enc_csn_id_coded,admit_time,feature_type,features,values
,<chr>,<dbl>,<chr>,<chr>,<chr>,<dbl>
1,JC29f8ad2,131274729058,2019-08-31 12:52:00+00:00,demo,ESI_i,3
2,JC29f8ad3,131278291027,2019-10-05 23:48:00+00:00,demo,ESI_i,3
3,JC29f8b9c,131266787806,2019-05-05 01:07:00+00:00,demo,ESI_i,2
4,JC29f8beb,131264387263,2019-03-15 03:35:00+00:00,demo,ESI_i,3
5,JC29f8beb,131279241689,2019-11-27 15:29:00+00:00,demo,ESI_i,3
6,JC29f8bef,131280937356,2019-11-30 10:35:00+00:00,demo,ESI_i,3


In [137]:
n2 <- features_all %>% select(feature_type, features) %>% distinct() %>% 
        group_by(features) %>% count(sort=TRUE) %>% filter(n==2)
head(n2)
tail(n2)

features,n
<chr>,<int>
ALB_0,2
ALB_1,2
ALB_10,2
ALB_2,2
ALB_3,2
ALB_4,2


features,n
<chr>,<int>
WBC_4,2
WBC_5,2
WBC_6,2
WBC_7,2
WBC_8,2
WBC_9,2


In [138]:
features_all %>% filter(features %in% n2$features) %>% distinct(feature_type)# group_by(features, feature_type) %>% count()

feature_type
<chr>
labs_results_train
vitals_train
labs_results_test
vitals_test
Lab
Microbiology
Procedures
Imaging


In [170]:
features_all %>% group_by(feature_type, features) %>% count(sort=TRUE) # 43356 vs 42752 

feature_type,features,n
<chr>,<chr>,<int>
demo,age,41366
demo,delta_ESI,41366
demo,delta_H,41366
demo,delta_W,41366
demo,English,41366
demo,ESI_i,41366
demo,gender,41366
demo,Height_i,41366
demo,insurance,41366


In [140]:
head(features_all)

,anon_id,pat_enc_csn_id_coded,admit_time,feature_type,features,values
,<chr>,<dbl>,<chr>,<chr>,<chr>,<dbl>
1,JC29f8ad2,131274729058,2019-08-31 12:52:00+00:00,demo,ESI_i,3
2,JC29f8ad3,131278291027,2019-10-05 23:48:00+00:00,demo,ESI_i,3
3,JC29f8b9c,131266787806,2019-05-05 01:07:00+00:00,demo,ESI_i,2
4,JC29f8beb,131264387263,2019-03-15 03:35:00+00:00,demo,ESI_i,3
5,JC29f8beb,131279241689,2019-11-27 15:29:00+00:00,demo,ESI_i,3
6,JC29f8bef,131280937356,2019-11-30 10:35:00+00:00,demo,ESI_i,3


In [171]:
nrow(features_all) #10529994 vs 10086505
summary(features_all$values)
# sum(is.na(features_all$first_label))

[1] 10086505

    Min.  1st Qu.   Median     Mean  3rd Qu.     Max. 
   0.000    1.000    1.000    3.276    2.000 1078.000 

In [172]:
features_all_year <- features_all %>% mutate(year = year(admit_time))

In [173]:
# coh5
features_all_year %>% group_by(year) %>% count()

year,n
<dbl>,<int>
2015,1995746
2016,1583617
2017,1558080
2018,2085851
2019,2312418
2020,550793


In [143]:
#coh4
features_all_year %>% group_by(year) %>% count()

year,n
<dbl>,<int>
2015,2037462
2016,1669123
2017,1638834
2018,2185662
2019,2421534
2020,577379


In [45]:
write.csv(features_all_year, file = file.path(featuredir, "2_9_coh4_features_all_long_year.csv"), row.names=FALSE) 

In [174]:
write.csv(features_all_year, file = file.path(featuredir, "2_9_coh5_features_all_long_year.csv"), row.names=FALSE) 

### TEST - OLD 
with 2_9_features_all_long.csv did not contain testbins with Conor's code, in SQL

In [106]:
yr <- read.csv(file.path(featuredir, '2_9_features_all_long_year.csv'))
yr0 <- yr %>% select(-year)
nrow(yr0)
nrow(features_all)
identical(yr0, features_all)
all.equal(yr0,features_all)

[1] 8258975

[1] 8258975

[1] FALSE

[1] "Component “anon_id”: 8257416 string mismatches"                          
[2] "Component “pat_enc_csn_id_coded”: Mean relative difference: 0.0005188065"
[3] "Component “admit_time”: 8258716 string mismatches"                       
[4] "Component “feature_type”: 6592107 string mismatches"                     
[5] "Component “features”: 8250486 string mismatches"                         
[6] "Component “values”: Mean relative difference: 1.638"

In [108]:
yr0 <- yr0 %>% arrange(anon_id, pat_enc_csn_id_coded, admit_time, feature_type, features, values)
ft <- features_all %>% arrange(anon_id, pat_enc_csn_id_coded, admit_time, feature_type, features, values)

In [109]:
identical(yr0, ft)
all.equal(yr0, ft)

[1] TRUE

[1] TRUE

In [100]:
joined <- inner_join(yr, features_all)

Joining, by = c("anon_id", "pat_enc_csn_id_coded", "admit_time", "feature_type", "features", "values")



In [101]:
summary(joined)

   anon_id          pat_enc_csn_id_coded  admit_time        feature_type      
 Length:8258975     Min.   :1.311e+11    Length:8258975     Length:8258975    
 Class :character   1st Qu.:1.312e+11    Class :character   Class :character  
 Mode  :character   Median :1.312e+11    Mode  :character   Mode  :character  
                    Mean   :1.312e+11                                         
                    3rd Qu.:1.313e+11                                         
                    Max.   :1.313e+11                                         
   features             values              year     
 Length:8258975     Min.   :   0.000   Min.   :2015  
 Class :character   1st Qu.:   1.000   1st Qu.:2016  
 Mode  :character   Median :   1.000   Median :2017  
                    Mean   :   3.721   Mean   :2017  
                    3rd Qu.:   2.000   3rd Qu.:2019  
                    Max.   :1078.000   Max.   :2020  

In [104]:
summary(yr)

   anon_id          pat_enc_csn_id_coded  admit_time        feature_type      
 Length:8258975     Min.   :1.311e+11    Length:8258975     Length:8258975    
 Class :character   1st Qu.:1.312e+11    Class :character   Class :character  
 Mode  :character   Median :1.312e+11    Mode  :character   Mode  :character  
                    Mean   :1.312e+11                                         
                    3rd Qu.:1.313e+11                                         
                    Max.   :1.313e+11                                         
   features             values              year     
 Length:8258975     Min.   :   0.000   Min.   :2015  
 Class :character   1st Qu.:   1.000   1st Qu.:2016  
 Mode  :character   Median :   1.000   Median :2017  
                    Mean   :   3.721   Mean   :2017  
                    3rd Qu.:   2.000   3rd Qu.:2019  
                    Max.   :1078.000   Max.   :2020  

In [105]:
summary(features_all)

   anon_id          pat_enc_csn_id_coded  admit_time        feature_type      
 Length:8258975     Min.   :1.311e+11    Length:8258975     Length:8258975    
 Class :character   1st Qu.:1.312e+11    Class :character   Class :character  
 Mode  :character   Median :1.312e+11    Mode  :character   Mode  :character  
                    Mean   :1.312e+11                                         
                    3rd Qu.:1.313e+11                                         
                    Max.   :1.313e+11                                         
   features             values        
 Length:8258975     Min.   :   0.000  
 Class :character   1st Qu.:   1.000  
 Mode  :character   Median :   1.000  
                    Mean   :   3.721  
                    3rd Qu.:   2.000  
                    Max.   :1078.000  